<a href="https://colab.research.google.com/github/Chanda92/CE888/blob/master/Lab5/Recommender_Features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hybrid Models for Recommendation Systems

Load Pandas, we are going to need it for manipulating data

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

Now load the data

In [0]:
user_ratings_df = pd.read_csv("https://raw.githubusercontent.com/Chanda92/CE888/master/Lab5/user_ratings.csv")
user_features_df = pd.read_csv("https://raw.githubusercontent.com/Chanda92/CE888/master/Lab5/user_features.csv")
item_features_df = pd.read_csv("https://raw.githubusercontent.com/Chanda92/CE888/master/Lab5/item_features.csv")

In [0]:
user_features_df["key"] = 0
user_features_df["user_id"] = range(user_features_df.shape[0])
item_features_df["key"] = 0
item_features_df["item_id"] = range(item_features_df.shape[0])

merged_df = pd.merge(user_features_df, item_features_df, left_index=True, on="key")
merged_df.drop(labels='key', axis=1, inplace=True)  # drop the "key" column


In [7]:
merged_df[["item_id", "user_id"]]

,item_id,user_id
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1


In [0]:
merged_df["rating"] = map(lambda ids: user_ratings_df.values[ids[1]][ids[2]], 
                          merged_df[["user_id", "item_id"]].itertuples())

train = merged_df.dropna()
test = merged_df[merged_df.isnull().any(axis=1)]


In [9]:
n_latent_factors = 2

user_ratings = user_ratings_df.values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

user_features = user_features_df.values
item_features = item_features_df.values

user_features = np.concatenate([np.ones(shape=(user_features.shape[0],1)), user_features], axis=1)
item_features = np.concatenate([np.ones(shape=(item_features.shape[0],1)), item_features], axis=1)

user_features_weights = np.random.random((user_ratings.shape[0], user_features.shape[1]))
item_features_weights = np.random.random((user_ratings.shape[1], item_features.shape[1]))

print(user_features)

[[1. 1. 0. 0. 0.]
 [1. 0. 1. 0. 1.]
 [1. 0. 0. 0. 2.]
 [1. 1. 0. 0. 3.]
 [1. 0. 1. 0. 4.]
 [1. 0. 0. 0. 5.]
 [1. 0. 0. 0. 6.]
 [1. 1. 0. 0. 7.]
 [1. 0. 1. 0. 8.]
 [1. 1. 0. 0. 9.]]


In [0]:
def predict_rating(user_id, item_id):
    """
    Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    
    user_score = user_features_weights[user_id].dot(user_features[user_id])
    item_score = item_features_weights[item_id].dot(item_features[item_id])
    #print(user_preference.dot(item_preference), user_score, item_score)
    return user_preference.dot(item_preference) + user_score + item_score


def train(user_id, item_id, rating, alpha=0.001, 
          latent_feature_weight_decay=0.1,
          user_weight_decay=0.01, item_weight_decay=0.0001):
    #print(item_id)
    prediction_rating = predict_rating(user_id, item_id)
    err = prediction_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * (latent_item_features[item_id] + latent_feature_weight_decay * latent_user_preferences[user_id])
    latent_item_features[item_id] -= alpha * err * (user_pref_values + latent_feature_weight_decay * latent_item_features[item_id])
    
    user_features_weights[user_id] -=alpha * err * (user_features[user_id] + user_weight_decay * user_features_weights[user_id])
    item_features_weights[item_id] -=alpha * err * (item_features_weights[item_id] + item_weight_decay * item_features_weights[item_id])
    
    return err


def sgd(iterations=30000):
    """ 
    Iterate over all users and all items and train for 
    a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
    mse = (np.array(error) ** 2).mean()          
    print(mse)


In [11]:
for _ in range(10): 
    sgd()  # Note decreasing values with increasing iterations

0.29222352245438804
0.28430866496628476
0.2815799804342149
0.28022497978751715
0.2794239888854356
0.278903923535801
0.27854802619317703
0.2782980038397404
0.2781215621140527
0.2779994635069937


In [12]:
predictions = np.zeros(shape=(latent_user_preferences.shape[0], latent_item_features.shape[0]))
print(user_features_weights)
print(item_features_weights)
for user_id in range(latent_user_preferences.shape[0]):
    for item_id in range(latent_item_features.shape[0]):
        predictions[user_id, item_id] =  predict_rating(user_id, item_id)
  

[[-1.591 -1.03   0.007  0.518  0.491]
 [ 2.181  0.591  2.05   0.484  2.102]
 [ 0.7    0.068  0.635  0.946  0.495]
 [ 0.396  0.464  0.586  0.675  0.753]
 [ 0.328  0.752  0.592  0.484 -0.262]
 [ 0.209  0.962  0.213  0.997  1.547]
 [ 0.562  0.693  0.374  0.898 -0.298]
 [ 0.913  0.544  0.46   0.31  -0.021]
 [ 0.743  0.363  0.837  0.91   0.559]
 [ 0.157  0.024  0.243  0.834 -0.235]]
[[1.669e+00 1.508e+00 6.904e-01 1.267e+00 2.632e-01]
 [1.718e-03 5.874e-03 5.759e-04 6.381e-04 2.062e-03]
 [3.080e+00 2.812e-01 1.836e+00 1.956e-01 1.781e+00]
 [8.051e-01 4.573e-01 1.476e+00 6.381e-01 4.627e-01]
 [6.841e-01 9.217e-01 1.694e+00 1.221e-01 1.591e+00]]


In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = user_ratings_df.columns

In [14]:
comparison_data


,The Call of Cthulhu,Frankenstein,Dracula,Neuromancer,Space Odyssey
0,"(8.0, 7.988769312688265)","(2.0, 2.0331059178827067)","(nan, -21.556801716426758)","(5.0, 4.984340066298795)","(4.0, 3.995841260268639)"
1,"(3.0, 2.8993335174330888)","(2.0, 2.5597096395434806)","(nan, 66.61761598558157)","(7.0, 6.618759199491814)","(7.0, 6.924508660659538)"
2,"(9.0, 9.043994066282899)","(nan, 4.365248648912166)","(7.0, 7.0031067483775296)","(8.0, 7.950925056957335)","(5.0, 5.000784634392805)"
3,"(nan, 8.966652561736929)","(nan, 4.931602814897886)","(7.0, 7.000004875951129)","(8.0, 7.9995761435540995)","(9.0, 8.999390146203964)"
4,"(nan, 3.8144240426910594)","(1.0, 0.5699088794618457)","(8.0, 7.988880467287889)","(3.0, 3.3694957272561092)","(7.0, 7.053711315503134)"
5,"(2.0, 2.0017958864966907)","(3.0, 2.994769750455926)","(5.0, 5.000030371791279)","(nan, 2.285171963790703)","(nan, 32.56843888249177)"
6,"(4.0, 4.309824626554118)","(2.0, 0.43620520997428114)","(nan, -3.825957124321511)","(2.0, 2.9985611530019933)","(7.0, 7.204939474521241)"
7,"(7.0, 6.760195589141224)","(1.0, 2.9075780215942326)","(2.0, 2.084670023902807)","(7.0, 5.663038317310063)","(9.0, 8.759966488141915)"
8,"(3.0, 3.101993499415629)","(3.0, 2.519163763070019)","(nan, 75.28593268758968)","(7.0, 7.289641437929996)","(3.0, 3.0612096083953997)"
9,"(4.0, 3.8917045449070615)","(nan, -0.14617546160344486)","(5.0, 4.983953477098835)","(3.0, 3.109861815913385)","(3.0, 2.996734253951449)"
